In [1]:
import tensorflow as tf
from tensorflow.contrib.model_pruning.python import pruning
from tensorflow.contrib.model_pruning.python.layers import layers
import keras
from utils import *

Using TensorFlow backend.


In [2]:
epochs = 3
batch_size = 256 # Entire training set
model_path_unpruned = "Model_Saves/Unpruned.ckpt"
model_path_pruned = "Model_Saves/Pruned.ckpt"
NUM_CLASS = 10

In [3]:
#Load Dataset
X_train, y_train = prepare_dataset(data_dir, 'train')
X_test, y_test = prepare_dataset(data_dir, 'test')
t = int(time.time())
#Labels to binary
y_train_binary = keras.utils.to_categorical(y_train,num_classes)
y_test_binary = keras.utils.to_categorical(y_test,num_classes)

Loading numpy
Loading numpy


In [4]:
batches = int(len(X_train) / batch_size)
print(batches)

195


In [ ]:
X_train.size

In [ ]:
# create the training datasets
dx_train = tf.data.Dataset.from_tensor_slices(X_train.astype(np.float32))
# apply a one-hot transformation to each label for use in the neural network
dy_train = tf.data.Dataset.from_tensor_slices(y_train).map(lambda z: tf.one_hot(z, NUM_CLASS))
# zip the x and y training data together and shuffle, batch etc.
train_dataset = tf.data.Dataset.zip((dx_train, dy_train)).shuffle(500).repeat().batch(batch_size)

In [ ]:
# do the same operations for the validation set
dx_test = tf.data.Dataset.from_tensor_slices(X_test.astype(np.float32))
dy_test = tf.data.Dataset.from_tensor_slices(y_test).map(lambda z: tf.one_hot(z, NUM_CLASS))
test_dataset = tf.data.Dataset.zip((dx_test, dy_test)).shuffle(500).repeat().batch(batch_size)

In [ ]:
train_iterator = train_dataset.make_initializable_iterator()
test_iterator = test_dataset.make_initializable_iterator()

In [ ]:
# # create general iterator
# iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
#                                                train_dataset.output_shapes)
# next_element = iterator.get_next()

In [ ]:
# # make datasets that we can initialize separately, but using the same structure via the common iterator
# training_init_op = iterator.make_initializer(train_dataset)
# validation_init_op = iterator.make_initializer(valid_dataset)

In [ ]:
# def nn_model(in_data):
#     bn = tf.layers.batch_normalization(in_data)
#     fc1 = tf.layers.dense(bn, 50)
#     fc2 = tf.layers.dense(fc1, 50)
#     fc2 = tf.layers.dropout(fc2)
#     fc3 = tf.layers.dense(fc2, 10)
#     return fc3

In [ ]:
# # create the neural network model
# logits = nn_model(next_element[0])

In [ ]:
# # add the optimizer and loss
# loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=next_element[1], logits=logits))
# optimizer = tf.train.AdamOptimizer().minimize(loss)
# # get accuracy
# prediction = tf.argmax(logits, 1)
# equality = tf.equal(prediction, tf.argmax(next_element[1], 1))
# accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
# init_op = tf.global_variables_initializer()

In [ ]:
# def train_input_fn(features, labels, batch_size):
#     """An input function for training"""
#     # Convert the inputs to a Dataset.
#     dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

#     # Shuffle, repeat, and batch the examples.
#     dataset = dataset.shuffle(1000).repeat().batch(batch_size)

#     # Return the dataset.
#     return dataset

In [ ]:
# # run the training
# epochs = 10
# with tf.Session() as sess:
#     sess.run(init_op)
#     sess.run(training_init_op)
#     for i in range(epochs):
#         l, _, acc = sess.run([loss, optimizer, accuracy])
#         if i % 50 == 0:
#             print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, l, acc * 100))
#     # now setup the validation run
#     valid_iters = 100
#     # re-initialize the iterator, but this time with validation data
#     sess.run(validation_init_op)
#     avg_acc = 0
#     for i in range(valid_iters):
#         acc = sess.run([accuracy])
#         avg_acc += acc[0]
#     print("Average validation set accuracy over {} iterations is {:.2f}%".format(valid_iters, (avg_acc / valid_iters) * 100))

In [5]:
tf.reset_default_graph()

image = tf.placeholder(name='images', dtype=tf.float32, shape=[None, 32, 32, 3])
label = tf.placeholder(name='fine_labels', dtype=tf.int32, shape=[None, 10])

# # Define the model
# layer1 = layers.masked_fully_connected(image, 300)
# layer2 = layers.masked_fully_connected(layer1, 300)
# logits = layers.masked_fully_connected(layer2, 10)

# Define the model
# layer1 = layers.masked_conv2d(image, 300, kernel_size=2)
# layer2 = layers.masked_conv2d(layer1, 300, kernel_size=2)
# logits = layers.masked_fully_connected(layer2, 10)

_=image
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-1')
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME',name='pool1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME', name='pool2')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'VALID')
_ = tf.layers.batch_normalization(_, name='norm3')
_ = layers.masked_conv2d(_, 192, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm4')
_ = layers.masked_conv2d(_, 10, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm5')
_ = tf.layers.average_pooling2d(_, (6,6), 1, name='avg_pool')
y = _
logits = tf.reshape(y,[tf.shape(y)[0],10])


# Create global step variable (needed for pruning)
global_step = tf.train.get_or_create_global_step()
reset_global_step_op = tf.assign(global_step, 0)

# Loss function
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=label))
#loss = tf.losses.softmax_cross_entropy(tf.one_hot(label, NUM_CLASS), logits)
loss = tf.losses.softmax_cross_entropy(label, logits)

# Training op, the global step is critical here, make sure it matches the one used in pruning later
# running this operation increments the global_step
train_op = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, global_step=global_step)

# Accuracy ops
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



In [6]:
# Get, Print, and Edit Pruning Hyperparameters
pruning_hparams = pruning.get_pruning_hparams()
print("Pruning Hyperparameters:", pruning_hparams)

# Change hyperparameters to meet our needs
pruning_hparams.begin_pruning_step = 0
pruning_hparams.end_pruning_step = 250
pruning_hparams.pruning_frequency = 1
pruning_hparams.sparsity_function_end_step = 250
pruning_hparams.target_sparsity = .5

# Create a pruning object using the pruning specification, sparsity seems to have priority over the hparam
p = pruning.Pruning(pruning_hparams, global_step=global_step) #sparsity=.5)
prune_op = p.conditional_mask_update_op()

# Create a saver for writing training checkpoints.
saver = tf.train.Saver()



Pruning Hyperparameters: [('begin_pruning_step', 0), ('block_height', 1), ('block_pooling_function', 'AVG'), ('block_width', 1), ('do_not_prune', ['']), ('end_pruning_step', -1), ('initial_sparsity', 0), ('name', 'model_pruning'), ('nbins', 256), ('pruning_frequency', 10), ('sparsity_function_begin_step', 0), ('sparsity_function_end_step', 100), ('sparsity_function_exponent', 3), ('target_sparsity', 0.5), ('threshold_decay', 0.9), ('use_tpu', False)]
INFO:tensorflow:Updating masks.


In [15]:
with tf.Session() as sess:

    # Uncomment the following if you don't have a trained model yet
    sess.run(tf.global_variables_initializer())

    # Train the model before pruning (optional)
    for epoch in range(epochs):
        for batch in range(batches):
            #batch_xs, batch_ys = train_iterator.get_next()
            batch_xs, batch_ys = sample_batch(X_train, y_train_binary, batch_size)
            #print(batch_xs.shape)
            #batch_xs = batch_xs.reshape(-1,28,28,1)
            sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

        # Calculate Test Accuracy every 10 epochs
        if epoch % 1 == 0:
            acc_print = 0
            for batch in range(batches):
                #batch_xt, batch_yt = test_iterator.get_next()
                batch_xt, batch_yt = sample_batch(X_test, y_test_binary, batch_size)
                print(batch_xt.shape)
                #batch_xt = batch_xt.reshape(-1,28,28,1)
                acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
            print("Un-pruned model step %d test accuracy %g" % (epoch, acc_print/batches))
        print(epoch)
    
    # Saves the model before pruning
    saver.save(sess, model_path_unpruned)
    
    acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images.reshape(-1,28,28,1), label: mnist.test.labels})
    print("Pre-Pruning accuracy:", acc_print)
    print("Sparsity of layers (should be 0)", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))

    

    # Resets the session and restores the saved model
    sess.run(tf.initialize_all_variables())
    saver.restore(sess, model_path_unpruned)

    # Reset the global step counter and begin pruning
    sess.run(reset_global_step_op)
    for epoch in range(epochs):
        for batch in range(batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(-1,28,28,1)
            # Prune and retrain
            sess.run(prune_op)
            sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

        # Calculate Test Accuracy every 10 epochs
        if epoch % 1 == 0:
            acc_print = 0
            for batch in range(batches):
                batch_xt, batch_yt = mnist.test.next_batch(batch_size)
                #print(batch_xt.shape)
                batch_xt = batch_xt.reshape(-1,28,28,1)
                acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
            
            print("Pruned model step %d test accuracy %g" % (epoch, acc_print/batches))
            print("Weight sparsities:", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))
        print(epoch) 
        
           # acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})

    # Saves the model after pruning
    saver.save(sess, model_path_pruned)

    # Print final accuracy
    #acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})
    print("Final accuracy:", acc_print)
    print("Final sparsity by layer (should be 0)", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))

(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 

(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 3)
(256, 32, 32, 

NameError: name 'mnist' is not defined

In [8]:
def create_batch(dataset,labels, batch_size):
    for index, offset in enumerate(range(0, dataset.shape[0], batch_size)):
        x_epoch, y_epoch = np.array(dataset[offset: offset + batch_size,:]), np.array(labels[offset: offset +  batch_size])
    return x_epoch, y_epoch

In [ ]:
X_train.shape[0]

In [13]:
def sample_batch(dataset,labels, batch_size):
    N = dataset.shape[0]
    indices = np.random.randint(N, size=batch_size)
    x_epoch = dataset[indices]
    y_epoch = labels[indices]
    return x_epoch, y_epoch

In [ ]:
epochs = 3
batch_size = 512 # Entire training set
model_path_unpruned = "Model_Saves/Unpruned.ckpt"
model_path_pruned = "Model_Saves/Pruned.ckpt"
NUM_CLASS = 10
tf.reset_default_graph()

# Import dataset
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batches = int(len(mnist.train.images) / batch_size)

# Define Placeholders
# image = tf.placeholder(tf.float32, [None, 28,28])
# label = tf.placeholder(tf.float32, [None, 10])

image = tf.placeholder(name='images', dtype=tf.float32, shape=[None, 28, 28, 1])
label = tf.placeholder(name='fine_labels', dtype=tf.int32, shape=[None,10])

# # Define the model
# layer1 = layers.masked_fully_connected(image, 300)
# layer2 = layers.masked_fully_connected(layer1, 300)
# logits = layers.masked_fully_connected(layer2, 10)

# Define the model
# layer1 = layers.masked_conv2d(image, 300, kernel_size=2)
# layer2 = layers.masked_conv2d(layer1, 300, kernel_size=2)
# logits = layers.masked_fully_connected(layer2, 10)

_=image
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-1')
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME',name='pool1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME', name='pool2')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'VALID')
_ = tf.layers.batch_normalization(_, name='norm3')
_ = layers.masked_conv2d(_, 192, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm4')
_ = layers.masked_conv2d(_, 10, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm5')
_ = tf.layers.average_pooling2d(_, (5,5), 1, name='avg_pool')
y = _
logits = tf.reshape(y,[tf.shape(y)[0],10])


# Create global step variable (needed for pruning)
global_step = tf.train.get_or_create_global_step()
reset_global_step_op = tf.assign(global_step, 0)

# Loss function
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=label))
#loss = tf.losses.softmax_cross_entropy(tf.one_hot(label, NUM_CLASS), logits)
loss = tf.losses.softmax_cross_entropy(label, logits)

# Training op, the global step is critical here, make sure it matches the one used in pruning later
# running this operation increments the global_step
train_op = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, global_step=global_step)

# Accuracy ops
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



In [ ]:
# Get, Print, and Edit Pruning Hyperparameters
pruning_hparams = pruning.get_pruning_hparams()
print("Pruning Hyperparameters:", pruning_hparams)

# Change hyperparameters to meet our needs
pruning_hparams.begin_pruning_step = 0
pruning_hparams.end_pruning_step = 250
pruning_hparams.pruning_frequency = 1
pruning_hparams.sparsity_function_end_step = 250
pruning_hparams.target_sparsity = .5

# Create a pruning object using the pruning specification, sparsity seems to have priority over the hparam
p = pruning.Pruning(pruning_hparams, global_step=global_step) #sparsity=.5)
prune_op = p.conditional_mask_update_op()

# Create a saver for writing training checkpoints.
saver = tf.train.Saver()



In [ ]:
with tf.Session() as sess:

    # Uncomment the following if you don't have a trained model yet
    sess.run(tf.global_variables_initializer())

    # Train the model before pruning (optional)
    for epoch in range(epochs):
        for batch in range(batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            #print(batch_xs.shape)
            batch_xs = batch_xs.reshape(-1,28,28,1)
            sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

        # Calculate Test Accuracy every 10 epochs
        if epoch % 1 == 0:
            acc_print = 0
            for batch in range(batches):
                batch_xt, batch_yt = mnist.test.next_batch(batch_size)
                #print(batch_xt.shape)
                batch_xt = batch_xt.reshape(-1,28,28,1)
                acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
            print("Un-pruned model step %d test accuracy %g" % (epoch, acc_print/batches))
        print(epoch)
    
    # Saves the model before pruning
    saver.save(sess, model_path_unpruned)
    
    acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images.reshape(-1,28,28,1), label: mnist.test.labels})
    print("Pre-Pruning accuracy:", acc_print)
    print("Sparsity of layers (should be 0)", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))

    

    # Resets the session and restores the saved model
    sess.run(tf.initialize_all_variables())
    saver.restore(sess, model_path_unpruned)

    # Reset the global step counter and begin pruning
    sess.run(reset_global_step_op)
    for epoch in range(epochs):
        for batch in range(batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(-1,28,28,1)
            # Prune and retrain
            sess.run(prune_op)
            sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

        # Calculate Test Accuracy every 10 epochs
        if epoch % 1 == 0:
            acc_print = 0
            for batch in range(batches):
                batch_xt, batch_yt = mnist.test.next_batch(batch_size)
                #print(batch_xt.shape)
                batch_xt = batch_xt.reshape(-1,28,28,1)
                acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
            
            print("Pruned model step %d test accuracy %g" % (epoch, acc_print/batches))
            print("Weight sparsities:", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))
        print(epoch) 
        
           # acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})

    # Saves the model after pruning
    saver.save(sess, model_path_pruned)

    # Print final accuracy
    #acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})
    print("Final accuracy:", acc_print)
    print("Final sparsity by layer (should be 0)", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))

In [ ]:
mnist

In [ ]:
batches